LET'S DO LAST STEPS FOR COMPLETING THIS PROJECT

In [1]:
import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_excel(r"D:\#PROJECTS\Crop Recommendation System for Indian Farmerss(c)\final_dataset.xlsx")

df = pd.DataFrame(dataset)

df

,Unnamed: 0,District,Month,Day,Predicted_Moisture_15cm,rainfall(mm)_full,temperature(°C)_full,previous crop
0,0,Ambala,1,1,7.672700,202.935536,20.879744,Garlic
1,1,Ambala,1,2,7.513698,226.655537,21.770462,Garlic
2,2,Ambala,1,3,7.539709,263.964248,23.004459,Wheat
3,3,Ambala,1,4,7.500415,242.864034,26.491096,Cabbage
4,4,Ambala,1,5,7.411865,262.717340,20.130175,Cabbage
...,...,...,...,...,...,...,...,...
3955,3955,Yamunanagar,11,26,10.472844,103.463655,25.616244,Urad (Black Gram)
3956,3956,Yamunanagar,11,27,10.533088,103.463655,25.616244,Garlic
3957,3957,Yamunanagar,11,28,10.935160,103.463655,25.616244,Urad (Black Gram)
3958,3958,Yamunanagar,11,29,10.940996,103.463655,25.616244,Masoor (Lentil)


In [6]:
data = pd.read_csv(r"D:\#PROJECTS\Crop Recommendation System for Indian Farmerss(c)\SoilMoisture_haryana_original.csv")
df = pd.DataFrame(data)

df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df['Year'] = df['Date'].dt.year
df['DayOfYear'] = df['Date'].dt.dayofyear
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.weekday
df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)

df = pd.get_dummies(df, columns=['District'], drop_first=False)

features = ['Year', 'Month', 'Month_sin', 'Month_cos', 'DayOfYear', 'Day', 
            'Weekday'] + [col for col in df.columns if 'District_' in col]
X = df[features]
y = df['Avg_smlvl_at15cm']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

model = lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.05, n_estimators=100)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

from sklearn.model_selection import GridSearchCV , RandomizedSearchCV

estimator_grid = {"num_leaves" : [31,40,50,60,70,100], "n_estimators" : [100,200,300,400] , "learning_rate" : [0.05,0.1,0.2,0.3]}

model_new = lgb.LGBMRegressor(objective='regression', 
                              learning_rate=0.2, n_estimators=400, num_leaves=100)
model_new.fit(X_train, y_train)

print(f"\n\n\n\n\n\n\n\nTRAINING SCORE OF NEW MODEL IS {model_new.score(X_train,y_train)*100}")
print(f"TESTING SCORE OF NEW MODEL IS {model_new.score(X_test,y_test)*100}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 356
[LightGBM] [Info] Number of data points in the train set: 6230, number of used features: 28
[LightGBM] [Info] Start training from score 12.002140
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000924 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 356
[LightGBM] [Info] Number of data points in the train set: 6230, number of used features: 28
[LightGBM] [Info] Start training from score 12.002140








TRAINING SCORE OF NEW MODEL IS 99.75354087353536
TESTING SCORE OF NEW MODEL IS 96.2898

In [8]:
def predict_moisture(district, year, month, day):
    # input for new data
    input_data = pd.DataFrame({
        'State': ['Haryana'],  
        'District': [district],
        'Month': [month],
        'Date': [f'{year}-{month:02d}-{day:02d}']
    })

    #finding features
    input_data['Date'] = pd.to_datetime(input_data['Date'])
    input_data['Year'] = input_data['Date'].dt.year
    input_data['DayOfYear'] = input_data['Date'].dt.dayofyear
    input_data['Day'] = input_data['Date'].dt.day
    input_data['Weekday'] = input_data['Date'].dt.weekday
    input_data['Month_sin'] = np.sin(2 * np.pi * input_data['Month'] / 12)
    input_data['Month_cos'] = np.cos(2 * np.pi * input_data['Month'] / 12)

    # One-hot encode District
    district_cols = [col for col in df.columns if 'District_' in col]
    input_data = pd.get_dummies(input_data, columns=['District'])
    for col in district_cols:
        if col not in input_data.columns:
            input_data[col] = 0

    # arranging
    features = ['Year', 'Month', 'Month_sin', 'Month_cos', 'DayOfYear', 'Day', 'Weekday'] + district_cols
    X_input = input_data[features]

    # Predict
    predicted = model_new.predict(X_input)
    return predicted[0]




In [12]:
#new data prediction
new_data = {
    "District": "Karnal",
    "Year": 2025,
    "Month": 11,
    "Day": 15,
    "temperature(°C)_full": 22,
    "rainfall(mm)_full": 30,
    "previous_crop": "Rice",
    "soil_type": "loamy"
}
predicted_moisture = predict_moisture(new_data["District"], new_data["Year"], new_data["Month"], new_data["Day"])
print(f"Predicted Soil Moisture Level at 15cm: {predicted_moisture}")

Predicted Soil Moisture Level at 15cm: 10.762549673082539


In [10]:
def recommend_crop(district, year, month, day, temp, rain, prev_crop, soil_type="loamy"):
    # Create a single-row dictionary like a DataFrame row
    row = {
        "District": district,
        "Year": year,
        "Month": month,
        "Day": day,
        "temperature(°C)_full": temp,
        "rainfall(mm)_full": rain,
        "previous_crop": prev_crop,
        "soil_type": soil_type
    }

    # --- Input Validation ---
    required_columns = ["District", "Year", "Month", "Day", "temperature(°C)_full", 
                        "rainfall(mm)_full", "previous_crop"]
    for col in required_columns:
        if col not in row or pd.isna(row[col]):
            return "Error: Missing required data"

    # --- Predict Moisture if Missing ---
    if pd.isna(row.get("Predicted_Moisture_15cm", None)):
        try:
            row["Predicted_Moisture_15cm"] = predict_moisture(
                district=row["District"],
                year=row["Year"],
                month=row["Month"],
                day=row["Day"]
            )
            # Ensure moisture is within realistic bounds (0–100%)
            row["Predicted_Moisture_15cm"] = max(0, min(100, row["Predicted_Moisture_15cm"]))
        except Exception as e:
            print(f"Prediction failed for district {row.get('District', 'Unknown')}: {e}")
            row["Predicted_Moisture_15cm"] = 10  # Default fallback moisture

    # --- Extract Inputs ---
    district = row["District"].lower().strip()
    month = int(row["Month"])
    temp = float(row["temperature(°C)_full"])
    rain = float(row["rainfall(mm)_full"])
    moist = float(row["Predicted_Moisture_15cm"])
    prev = row["previous_crop"].lower().strip()
    soil_type = row.get("soil_type", "loamy").lower().strip()  # Default to loamy if not provided

    # --- Input Validation for Numeric Values ---
    if temp < -10 or temp > 50:
        return "Error: Invalid temperature"
    if rain < 0:
        return "Error: Invalid rainfall"
    if moist < 0 or moist > 100:
        return "Error: Invalid moisture value"

    # --- Agro-climatic Zone ---
    north_districts = ["yamunanagar", "ambala", "kurukshetra", "karnal"]
    central_districts = ["panipat", "rohtak", "jind", "kaithal", "sonipat"]
    if district in north_districts:
        zone = "north"
    elif district in central_districts:
        zone = "central"
    else:
        zone = "south"

    # --- Crop Suitability Scoring ---
    crop_scores = {
        "Wheat": 0, "Mustard": 0, "Barley": 0, "Gram": 0, "Rice": 0,
        "Maize": 0, "Cotton": 0, "Bajra": 0, "Sugarcane": 0,
        "Moong (Green Gram)": 0, "Sunflower": 0, "Vegetables": 0
    }

    # --- Seasonal Logic ---
    if month in [11, 12, 1, 2, 3]:  # Rabi Season (Winter)
        if any(crop in prev for crop in ["rice", "paddy", "urad"]):
            crop_scores["Wheat"] += 30
        if any(crop in prev for crop in ["cotton", "maize"]):
            crop_scores["Mustard"] += 20
            crop_scores["Gram"] += 15
        if moist < 8 and rain < 50:
            crop_scores["Mustard"] += 25
        if 8 <= moist <= 12 and 18 <= temp <= 26:
            crop_scores["Wheat"] += 25
        if temp < 22 and rain < 40:
            crop_scores["Barley"] += 20
        if zone == "south":
            crop_scores["Gram"] += 20
        if soil_type in ["loamy", "sandy loam"]:
            crop_scores["Wheat"] += 10
            crop_scores["Barley"] += 10
        if soil_type == "clayey":
            crop_scores["Gram"] += 10

    elif month in [6, 7, 8, 9, 10]:  # Kharif Season (Monsoon)
        if any(crop in prev for crop in ["wheat", "barley"]):
            if rain > 120 and moist > 10:
                crop_scores["Rice"] += 30
            if zone == "south":
                crop_scores["Cotton"] += 25
            else:
                crop_scores["Maize"] += 20
        if any(crop in prev for crop in ["mustard", "gram"]):
            if zone == "north":
                crop_scores["Rice"] += 25
            elif zone == "south":
                crop_scores["Bajra"] += 20
            else:
                crop_scores["Maize"] += 20
        if rain < 80 and temp > 30:
            crop_scores["Bajra"] += 20
        if rain > 150 and zone == "north":
            crop_scores["Sugarcane"] += 25
        if soil_type in ["loamy", "clayey"]:
            crop_scores["Rice"] += 15
            crop_scores["Sugarcane"] += 10
        if soil_type == "sandy":
            crop_scores["Bajra"] += 15
            crop_scores["Cotton"] += 10

    elif month in [4, 5]:  # Zaid Season (Summer)
        if temp > 30 and rain < 80:
            crop_scores["Moong (Green Gram)"] += 25
        if rain > 100:
            crop_scores["Maize"] += 20
        if moist > 12:
            crop_scores["Vegetables"] += 25
        if soil_type in ["loamy", "sandy loam"]:
            crop_scores["Sunflower"] += 15
            crop_scores["Moong (Green Gram)"] += 10
        if soil_type == "clayey":
            crop_scores["Vegetables"] += 10

    # --- Select Crop with Highest Score ---
    max_score = max(crop_scores.values())
    if max_score == 0:
        return "Vegetables"  # Default fallback
    recommended_crop = max(crop_scores, key=crop_scores.get)
    return recommended_crop

    row_df = pd.DataFrame([row])
    recommended_crop = recommend_crop_auto(row_df.iloc[0])
    return recommended_crop

# --- Farmer-friendly wrapper ---
def my_crop(district, year, month, day, temp, rain, prev_crop, soil_type="loamy"):
    crop = recommend_crop(district, year, month, day, temp, rain, prev_crop, soil_type)
    print(f"Recommended Crop for {district.title()} on {day}-{month}-{year}: {crop}")
    return crop








predict new data










In [11]:
my_crop("ambala", 2023, 1, 31, 2, 100, "Wheat")  #(district, year, month, day, temp, rain, prev_crop)

#district is string
#year is int
#month is int
#day is int
#temp is float
#rain is float
#prev_crop is string

Recommended Crop for Ambala on 31-1-2023: Wheat


'Wheat'